# pig strategy classification

In [181]:
import random
import math
import numpy as np
from numba import njit

In [182]:
@njit
def own_should_roll(difficulties0, difficulties1, turn_totals0, turn_totals1, scores0, scores1, num_turns_held0, num_turns_held1, own):
    difficulties = [difficulties0, difficulties1]
    turn_totals = [turn_totals0, turn_totals1]
    scores = [scores0, scores1]
    num_turns_held = [num_turns_held0, num_turns_held1]
    own_difficulty = difficulties[own]
    
    # keep pace and end race
    if own_difficulty == 4:
        if scores[own] + turn_totals[own] >= 100:
            return False
        if scores[1 - own] >= 71 or scores[own] >= 71 or turn_totals[own] < 21 + round((scores[1 - own] - scores[own]) / 8):
            return True
        return False
    
    # Score Base, Keep Pace, and End Race
    elif own_difficulty == 3:
        if scores[1 - own] >= 69 or scores[own] >= 69:
            return True
        else:
            if turn_totals[own] >= max(19, scores[1 - own]):
                return False
            else:
                return True
      
    # 4 scoring turns
    elif own_difficulty == 2:
        if turn_totals[own] >= math.floor((100 - scores[own]) / (4 - num_turns_held[own])):
            return False
        else:
            return True
    
    # hold at 25 or 100
    elif own_difficulty == 1:
        if turn_totals[own] >= 25 or scores[own] + turn_totals[own] >= 100:
            return False
        else:
            return True
    
    # hold at 20 or 100
    elif own_difficulty == 0: 
        if turn_totals[own] >= 20 or scores[own] + turn_totals[own] >= 100:
            return False
        else:
            return True

@njit
def simulate_game(a,b):
    difficulties = [a,b]
    scores = [0,0]
    current_player = 0
    num_turns_held = [0,0]
    while scores[0] < 100 and scores[1] < 100:
#         print(f"player {current_player}")
        roll = 0
        turn_totals = [0,0]
        while roll != 1 and turn_totals[current_player] + scores[current_player] < 100:
            
            
            
            
            
            if own_should_roll(difficulties[0], difficulties[1], turn_totals[0], turn_totals[1], scores[0], scores[1], num_turns_held[0], num_turns_held[1], current_player):
                roll = random.randint(1,6)
                if roll == 1:
                    turn_totals[current_player] = 0
                else:
                    turn_totals[current_player] += roll
#                 print(f"Roll: {roll} ({turn_totals[current_player]})")
            else:
                num_turns_held[current_player] += 1
                break
#         print(f"turn total: {turn_totals[current_player]}")
        scores[current_player] += turn_totals[current_player]
#         print(scores)
#         print(num_turns_held)
        current_player = 1 - current_player

    return 1 - current_player

In [183]:
competition_data = np.zeros((5,5))

t = 100_000

for a in range(5):
    for b in range(5):
        s = 0
        for _ in range(t):
            s += (0 == simulate_game(a,b))
        competition_data[a][b] = s/t

In [184]:
competition_data

array([[0.53312, 0.52193, 0.51904, 0.56537, 0.4868 ],
       [0.54349, 0.52898, 0.52723, 0.56244, 0.50764],
       [0.54747, 0.53313, 0.53465, 0.57077, 0.51407],
       [0.50125, 0.49566, 0.49514, 0.5337 , 0.47774],
       [0.57576, 0.55445, 0.54878, 0.5853 , 0.53043]])